# Film preperation using particle deposition.

This tutorial demonstrates particle film generation using ballistic deposition.

#### Current maintainer
- Stefan Endres (s.endres@iwt.uni-bremen.de)

#### Previous authors:     
- Valentine Baric 
- Lutz Mädler$^*$ (lmaeder@iwt.uni-bremen.de)
- Norbert Riefler 

#### Literature: 
##### Film generation primary sources:

- Mädler et al. (2006), Nanotechnology 17, 3783-4795
- Norbert, R. and Mädler. L. (2010) Journal of Nanoparticle Research 12.3 : 853-863.

##### Secondary:

- Filippov et al. (2000), Journal of Colloid and Interface Science 229, 261-273
- Ermak and Buckholz (1980), Journal of Computational Physics 35. 169-182
- Chan and Dahneke (1981), J. Appl. Phys. 52 3106-10

In [1]:
## Imports
# Std. library
import sys, random, os, datetime, warnings, socket, argparse
import multiprocessing as mp
import numpy as np
# Dependencies:
from scipy import spatial, stats
import shutil, fcntl, termios, struct
# Local imports:
from main_function import *
import functions as f
from create_aggregate import create_aggregate as SA
from create_aggregate_CCA import create_aggregate as CCA
import debugging as debug
from classes_agg import Aggregate as Aggregate
from classes_agg import Particle as Particle
import write_output
import terminal_print
import statistics
warnings.filterwarnings('error')
if sys.stdout.isatty():
    Redirection = 0
    COLS = struct.unpack('hh',  fcntl.ioctl(sys.stdout, termios.TIOCGWINSZ, '1234'))[1] -1
else:
    COLS = 60
    Redirection = 1

# Parameters

Next we import the parameters for the film generation, the `dda_template.py` file can be modified to add custom parameters, alteratively the desired imports can be modified in this notebook.

In [2]:
# The inputfile has to be copied to the folder of the source, otherwise it cannot be imported (shutil.copy2) (only if they are not already in the same folder)
import dda_template as params
source_file = os.getcwd()
PID = os.getpid() 
params_file = 'dda_template.py'

# Initiate parameters:
init_params(params_file, source_file)
# Example parameters that can be modified:
params.N_tot = 20#0000  # Maximum number of primary particles (PP)  (test with low number)
params.experiment  # The name of this experiment
params.output_folder  # Folder for output data
params.T  # Gas temperature
params.D_f  # Fractal dimension
k_f = params.k_f  # Prefractor for D_f
epsilon = params.epsilon  # Minimum distance between two PP
delta = params.delta  # Maximum distance between two PP
deformation = params.deformation  # Deformation factor
params.rho  # Density of the particle
params.dt  # Timestep
Height_above = params.Height_above  # Initialization hight above the highest particle

# Parrallelization:
num_cores = 1  # Specify the number of CPUs used for parallel computing
jobserver = mp.Pool(processes = num_cores)
if num_cores == -1:
    num_cores = mp.cpu_count()
elif num_cores <= 0 or num_cores > mp.cpu_count():
    num_cores = mp.cpu_count()
    print("""Invalid number of CPUs
        ==> Maximum number of CPU used ('%i')""" %num_cores)
    
jobserver = mp.Pool(processes = num_cores)

# Algorithmic controls:
k_B = 1.3806488e-23         # Boltzmann-Constant in J/K
start = datetime.datetime.now().replace(microsecond=0) # Starting time
cwd = os.getcwd()           # Current directory
RESTART = False
random.seed(params.custom_seed_traj)  # set a new seed for random numbers

Exception ignored in: <function Pool.__del__ at 0x750c00ddb600>
Traceback (most recent call last):
  File "/home/endres/anaconda3/envs/ddg/lib/python3.11/multiprocessing/pool.py", line 268, in __del__
    _warn(f"unclosed running multiprocessing pool {self!r}",


In [3]:
archive_folder = './archive'

#current_date = now.strftime("%Y%m%d")
write_lammpstrj = output_trj(params)

Exception ignored in: <_io.FileIO name='./dda/log.dda' mode='wb' closefd=True>
Traceback (most recent call last):
  File "/tmp/ipykernel_142269/1220664847.py", line 4, in <module>


# Film generation

In [4]:
results, aggs, N_PP_per_Agg = generate_film(params, source_file, num_cores, jobserver)


************************************************************
Process data
	Experiment:		dda
	Starting Time:		2025-03-14 14:09
	PID:			142269
	CPUs:			1

************************************************************
System parameters
	Pe:			1.00e+00
	Box size:		280
	Primary Particles:	20
	Cluster per Aggregate:	6
	Aggregates:		1
	Particle Number Dist.:	none
	Particle Size Dist.:	lognorm
	Median radius (nm):	4.479e+00
	Sigma:			3.700e-01
	Neighbor Mode:		2
	Friction Model:	Chan&Dahneke

************************************************************
Modules
	Calculate Coordination (2)
		Threshold: Delta (2)
	Calculate Percolation (3)
		Threshold: One rp (1)

************************************************************
Outputfiles
	Default outputdata
	log file (1)
	LIGGGHTS hybrid granular/molecular data file (2)
	LIGGGHTS granular data file (3)
	VTK Output for paraview (5)
	Histogram with primary particle size distibution (7)
	Histogram with primary particle number distibution (8)

********

In [5]:
pt, maximal_radius = global_pp_indexing(aggs)

In [6]:
write_output.N_histogram(N_PP_per_Agg, params.output_folder, params.experiment, params)

# Particle Transport
At first the velocity is calculated according to the Langevin
equation of motion. This equation is solved according to Ermak and
Buckholz, 1980 with a random fluctuating force (B1/B2) and an applied force
(only relevant for the z-direction (force_z)). According to this velocity
the displacement is calculated and the aggregate is moved. Each timestep (v)
is computed on basis of the previous step (v0)


In [7]:
max_z = particle_transport(aggs, pt, params, maximal_radius, write_lammpstrj)

#max_z = 2


##### Particle Transport calculation initiated



ValueError: setting an array element with a sequence.

# Post processing

In [ ]:
write_lammpstrj.close()  # Note, if you want to re-run the cells above this needs to be re-opened

# Generate .trj and .vtk files in the output_folder    
post_proc(params, pt, aggs, source_file, max_z)

# Film properties and visualisations

In the `functions.py` file functions for computing the film `packing_density`, `coordination` number and `percolation` of the film are found. The `.trj` and `.vtk` files can be visualised. The cell below allows for visualisation of the particle film using polyscope which can be downloaded easily from PyPi using `$ pip install polyscope`:


In [ ]:
from classes.particlelayer import *
import numpy as np
import json
import polyscope as ps

file  = './dda/dda.trj'
layer = particlelayer(file, nodescription=False).layer

# Shape
layer._data.shape

# Extract positions in space
points = np.zeros([layer.data.shape[0], 3])
points[:, 0] = layer.data[:, layer.lib['x']]
points[:, 1] = layer.data[:, layer.lib['y']]
points[:, 2] = layer.data[:, layer.lib['z']]

# Extract the radius
radii  = np.zeros(layer.data.shape[0])
radii = layer.data[:, layer.lib['r']]

# Plot in polyscope
ps.init()
ps.set_up_dir("z_up")
ps_cloud = ps.register_point_cloud("Particles", points)
ps_cloud.add_scalar_quantity("Particle radii", radii)
ps_cloud.set_point_radius_quantity("Particle radii", autoscale=False)
ps_cloud.set_color((1.0, 1.0, 1.0))
ps.show()

Explanation of main classes and functions:
#### AGGREGATE-CLASS
    
    Additional to this main program there are some more module files:
    Whenever a property of an aggregate is needed you will see something like:
    aggs[k].r
    This means that in aggs there are all aggregates stored and "k" is the
    index of the aggregate. ".r" is the property of the aggregate, in this case
    r shows all the radii of the PP within this aggregate.
    All properties and functions which start like this (aggs[k].) are found in classes.py

#### PARTICLE-CLASS
    
    Same as above for AGGREGATE. The abbreviation is pt[k]. It is also found in classes.py

#### FUNCTIONS
    
    Functions which are not specific for one aggregate are in the document functions.py
    So if there is a function like "f.rank_z()" you'll find it there.